## Side Notes for Lesson 3

### 1. Simple word embeddings with nltk and gensim

Following: https://github.com/nltk/nltk/blob/develop/nltk/test/gensim.doctest

In [1]:
import nltk
from nltk.corpus import brown
from nltk.data import find
from google.cloud import bigquery
import gensim
bqclient = bigquery.Client()
import numpy as np
import pandas as pd

/home/ejhaselden/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# pip install gensim

Define a couple of helper functions for cosine similarities: one deriving similarity between two words in the context of a model, the other for two vectors directly:

https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92

https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

https://jalammar.github.io/illustrated-word2vec/

https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [3]:
query = (
    """
    SELECT * FROM `w266-313114.final_project_clone.reconstructed_assets`
    """
)
result = bqclient.query(query).to_dataframe()
result

,asset_id,cn
0,45248253,"film industry, arts culture and entertainment,..."
1,49642909,"775637829, Basketball, Sport, Basketball Moves..."
2,46930785,"775635494, Audio Equipment, Spokesperson, Publ..."
3,31904237,"775578330, 10.21.2020 CMT Awards Fan Viewing P..."
4,50434474,"775641976, Green, Eyelash, Hair, Font, Dress, ..."
...,...,...
779397,40965793,BFfulltakes_FTP
779398,36318105,BFfulltakes_FTP
779399,33632646,BFfulltakes_FTP
779400,36318332,BFfulltakes_FTP


In [4]:
# result_small = result.iloc[0:7000,:]
# result_small

In [5]:
expanded = result['cn'].str.split(', ', 20, expand=True)

In [6]:
expanded

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,film industry,arts culture and entertainment,Choreography,Dance,celebrities,775621438,awards ceremony,BFselects_FTP,Entertainment,Fashion Design,...,None,None,None,None,None,None,None,None,None,None
1,775637829,Basketball,Sport,Basketball Moves,Sports Uniform,Footwear,Contact Sport,ozteam,Justinian Jessup,Active Shorts,...,Basketball Court,Player,Shorts,Field House,Sports Equipment,Basketball Player,None,None,None,None
2,775635494,Audio Equipment,Spokesperson,Public Address System,Keyboard,Microphone,Music Artist,Entertainment,BFfulltakes_FTP,Microphone Stand,...,Artist,Purple,Singing,Musician,Celebrities,Performing Arts,Arts Culture and Entertainment,None,None,None
3,775578330,10.21.2020 CMT Awards Fan Viewing Party_Kempin,Darkness,Music,Performance,Sky,Fashion,Celebrities,BFfulltakes_FTP,Arts Culture and Entertainment,...,None,None,None,None,None,None,None,None,None,None
4,775641976,Green,Eyelash,Hair,Font,Dress,Arm,BFfulltakes_FTP,Arts Culture and Entertainment,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779397,BFfulltakes_FTP,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
779398,BFfulltakes_FTP,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
779399,BFfulltakes_FTP,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
779400,BFfulltakes_FTP,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [7]:
# result = result.astype(str)
result['asset_id'] = 'assetnum' + result['asset_id'].astype(str)
embed_df = pd.concat([expanded, result.asset_id], axis=1)
embed_df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,asset_id
0,film industry,arts culture and entertainment,Choreography,Dance,celebrities,775621438,awards ceremony,BFselects_FTP,Entertainment,Fashion Design,...,None,None,None,None,None,None,None,None,None,assetnum45248253
1,775637829,Basketball,Sport,Basketball Moves,Sports Uniform,Footwear,Contact Sport,ozteam,Justinian Jessup,Active Shorts,...,Player,Shorts,Field House,Sports Equipment,Basketball Player,None,None,None,None,assetnum49642909
2,775635494,Audio Equipment,Spokesperson,Public Address System,Keyboard,Microphone,Music Artist,Entertainment,BFfulltakes_FTP,Microphone Stand,...,Purple,Singing,Musician,Celebrities,Performing Arts,Arts Culture and Entertainment,None,None,None,assetnum46930785
3,775578330,10.21.2020 CMT Awards Fan Viewing Party_Kempin,Darkness,Music,Performance,Sky,Fashion,Celebrities,BFfulltakes_FTP,Arts Culture and Entertainment,...,None,None,None,None,None,None,None,None,None,assetnum31904237
4,775641976,Green,Eyelash,Hair,Font,Dress,Arm,BFfulltakes_FTP,Arts Culture and Entertainment,None,...,None,None,None,None,None,None,None,None,None,assetnum50434474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779397,BFfulltakes_FTP,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,assetnum40965793
779398,BFfulltakes_FTP,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,assetnum36318105
779399,BFfulltakes_FTP,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,assetnum33632646
779400,BFfulltakes_FTP,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,assetnum36318332


In [8]:
lol_aid = embed_df.values.tolist()
# cns = result.asset_id
# lol = cns.values.tolist()
print(len(lol_aid))
lol_aid

779402


[['film industry',
  'arts culture and entertainment',
  'Choreography',
  'Dance',
  'celebrities',
  '775621438',
  'awards ceremony',
  'BFselects_FTP',
  'Entertainment',
  'Fashion Design',
  'Performing Arts',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'assetnum45248253'],
 ['775637829',
  'Basketball',
  'Sport',
  'Basketball Moves',
  'Sports Uniform',
  'Footwear',
  'Contact Sport',
  'ozteam',
  'Justinian Jessup',
  'Active Shorts',
  'Ball',
  'Basketball Court',
  'Player',
  'Shorts',
  'Field House',
  'Sports Equipment',
  'Basketball Player',
  None,
  None,
  None,
  None,
  'assetnum49642909'],
 ['775635494',
  'Audio Equipment',
  'Spokesperson',
  'Public Address System',
  'Keyboard',
  'Microphone',
  'Music Artist',
  'Entertainment',
  'BFfulltakes_FTP',
  'Microphone Stand',
  'Music',
  'Artist',
  'Purple',
  'Singing',
  'Musician',
  'Celebrities',
  'Performing Arts',
  'Arts Culture and Entertainment',
  None,
  N

In [9]:
from gensim import models
model = models.Word2Vec(lol_aid, vector_size=10, window=5, min_count=1, workers=4)
# model = models.Word2Vec(lol_aid)

In [10]:
model.wv.similarity('Blue', 'Violet')

0.9359877

In [11]:
model.wv.most_similar('Blue')

[('assetnum35810010', 0.9633541107177734),
 ('Red', 0.9626696109771729),
 ('Fun', 0.9591847658157349),
 ('assetnum51883084', 0.9589256644248962),
 ('Lighting', 0.9570203423500061),
 ('Pink', 0.9549674987792969),
 ('assetnum50854354', 0.9542205929756165),
 ('assetnum38415027', 0.952082097530365),
 ('assetnum49680601', 0.9500846266746521),
 ('assetnum31891548', 0.947597324848175)]

In [12]:
idx = model.wv.index_to_key
n = [x for x in idx if x is not None]
n.sort()
print(n)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
embed_df[embed_df.asset_id == 'assetnum35810010']

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,asset_id
208665,Celebrities,Performance,775589786,Fur,Event,Music,Fashion,BFfulltakes_FTP,Performing Arts,Arts Culture and Entertainment,...,None,None,None,None,None,None,None,None,None,assetnum35810010


In [16]:
model.wv.most_similar('assetnum35810010')

[('assetnum50854354', 0.9734025597572327),
 ('Blue', 0.9633541703224182),
 ('assetnum2020098', 0.9486265778541565),
 ('Green', 0.9483153223991394),
 ('assetnum35811234', 0.9457129836082458),
 ('assetnum51230823', 0.9453953504562378),
 ('assetnum51193971', 0.9451467990875244),
 ('assetnum32347623', 0.9436769485473633),
 ('Trousers, Blazer, Event, Music, Stage', 0.9395998120307922),
 ('assetnum1834298', 0.939491868019104)]

In [17]:
target = 'assetnum35810010'
for neighbor in model.wv.most_similar(target):
    print(neighbor[0])
    print(embed_df[embed_df.asset_id == neighbor[0]])

assetnum50854354
                 0             1                2                3  \
753551  Microphone  Music Artist  Awards Ceremony  Performing Arts   

                                     4                      5      6  \
753551  Arts Culture and Entertainment  Public Address System  Event   

                7                8      9  ...        12           13  \
753551  775649785  Audio Equipment  Music  ...  Musician  Celebrities   

                   14    15    16    17    18    19    20          asset_id  
753551  Entertainment  None  None  None  None  None  None  assetnum50854354  

[1 rows x 22 columns]
Blue
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, asset_id]
Index: []

[0 rows x 22 columns]
assetnum2020098
                     0        1              2           3     4     5     6  \
695619  KICKR HEADWIND  Product  Product Shots  Technology  None  None  None   

           7     8     9  ...    12    13    1

### 2. Simple BOW Classification using Word Embeddings in Keras

This section roughly implements the model on slides 41 in a toy setting.

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
# from tensorflow.keras.models import Model
# import tensorflow.keras.backend as K

Ok, now we know the number of words that have an embedding. Let's build the embedding matrix from the model:

In [ ]:
# EMBEDDING_DIM = len(model['university'])      # we know... it's 300

# # initialize embedding matrix and word-to-id map:
# embedding_matrix = np.zeros((len(model.vocab.keys()) + 1, EMBEDDING_DIM))       
# vocab_dict = {}

# # build the embedding matrix and the word-to-id map:
# for i, word in enumerate(model.vocab.keys()):
#     embedding_vector = model[word]
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         embedding_matrix[i] = embedding_vector
#         vocab_dict[word] = i



In [ ]:
# MAX_SEQUENCE_LENGTH = 5  # Keras' embedding layer expects a specific input length. Padding is often needed here.

# embedding_layer = Embedding(embedding_matrix.shape[0],
#                             embedding_matrix.shape[1],
#                             embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
#                             input_length=MAX_SEQUENCE_LENGTH,
#                             trainable=False)

In [ ]:
# try:
#     del tf_model
# except:
#     pass

Note the 'trainable=False' flag...

Now let's build the model, again as a **Sequential Model**: 

In [ ]:
# tf_model = tf.keras.Sequential()

# tf_model.add(embedding_layer)                                        # embedding layer
# tf_model.add(tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1)))    # average of embedding vectors
# tf_model.add(Dense(100, activation='relu'))                          # hidden layer
# tf_model.add(Dense(1, activation='sigmoid'))                         # classification layer

Let's see whether our dimension discussion was correct. Print a model summary:

In [ ]:
# tf_model.summary()

Like last week... let's compile the model. I.e, define optimizer, loss function, etc.

In [ ]:
# tf_model.compile(optimizer='adam', loss='BinaryCrossentropy')

Almost there... let's create some fake training and test data.

In [ ]:
# train_sentences = ['this is really absolutely great', 'this is really absolutely terrible']
# train_labels = [[1], [0]]

# test_sentences = ['never seen anything this stupid', 'never seen anything this fantastic']
# test_labels = [[0], [1]]

... and then do some formatting gymnastics:

In [ ]:
# def sents_to_ids(sentences):
#     """
#     converting a list of strings to a list of lists of word ids
#     """
#     text_ids = []
#     for sentence in sentences:
#         example = []
#         for word in sentence.split(' '):
#             example.append(vocab_dict[word])
#         text_ids.append(example)

#     return  text_ids   


# train_input = np.array(sents_to_ids(train_sentences))
# train_labels = np.array(train_labels)

# test_input = np.array(sents_to_ids(test_sentences))
# test_labels = np.array(test_labels)

So the model input are word ids in the vocab:

In [ ]:
# train_input

Next: let's get the start predictions. Should be random-ish. Are they?

In [ ]:
# print(tf_model.predict(train_input))
# print(tf_model.predict(test_input))

Yup, looks quite random.

Finally... let's train!

In [ ]:
# tf_model.fit(train_input, train_labels, validation_data=(test_input, test_labels), epochs=1)
# tf_model.fit(train_input, train_labels, validation_data=(test_input, test_labels), epochs=150, verbose=0)
# tf_model.fit(train_input, train_labels, validation_data=(test_input, test_labels), epochs=1)

Look's good!

What are train & test predictions now?

In [ ]:
# tf_model.predict(test_input)